## Introduction:
This script is a project I worked on recently in connection with an outage of the time-keeping software Kronos, that you may have heard about.  They were ransomed and our entire department lost access to the ability to keep time.  The managers of my department created the following solution:
a google form was created and sent to all the employees.  We were expected to open the form whenever we were clocking in or out and enter our employee ID.  This was all. It resulted in a spreadsheet with the first column containing a timestamp, and the second column containing the employee ID.  This was used by about 60 employees for two weeks, resulting in hundreds of lines in the spreadsheet.  I recognized that there would be no reasonably simple method using excel to categorize this information and get it in a readable format, so I created the following code:

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
date_parser = pd.to_datetime
pd.set_option('display.max_rows', None)

In [ ]:
# our time clocking system rounds to the nearest 15 minutes by default, and this method takes a time and does that rounding.
def rounder (dt):
    mm = int(dt.minute)
    
    if mm < 8:
        r=0
    elif mm > 7 and mm < 23:
        r = 15
    elif mm > 22 and mm <38:
        r = 30
    elif mm > 37 and mm <53:
        r = 45
    elif mm > 52:
        r = 60
        
    if r < mm:
        return dt + (r-mm)
    elif r == mm:
        return dt + 0
    else:
        return dt + (r-mm) 

def files(a):
    pass

In [ ]:
# while True:
#     try:
#         inputfile = input('Enter raw data file name with file type (e.g. sUdo Kronos (UKG) Temporary Replacement.csv): ')
#         df = pd.read_csv(inputfile,parse_dates=['timestamp'])
# except:
#     print('File not found')
# inputst = input('Enter pay period start date (mm/dd/yyyy): ')
# inputen = input('Enter pay period end date, last date inclusive (mm/dd/yyyy): ')
# while inputreq > 3 & inputreq < 1:
#     intputreq = int(input('Enter 1 for error report, 2 for total hour report, and 3 for both: '))
#     break

Enter raw data file name with file type (e.g. sUdo Kronos (UKG) Temporary Replacement.csv): bob
File not found


KeyboardInterrupt: 

In [ ]:
# read the file
df = pd.read_csv('kronos12-15-21_12-27-21.csv',parse_dates=['timestamp'])

In [ ]:
# convert time to minutes, use the above function to round all timestamps, make timestamps srtrings, and convert to pandas datetime format
df = df.sort_values(['Please enter your uNID','timestamp'], ignore_index=True)
df['timestamp'] = df['timestamp'].dt.to_period('min')
df['timestamp'] = df['timestamp'].apply(lambda row :rounder(row))
df['timestamp'] = df['timestamp'].astype(str)
df['timestamp'] = pd.to_datetime(df['timestamp'])
# startt = pd.to_datetime(inputst)
# endt = pd.to_datetime('12/28/2021')
# df = df[(df['timestamp'] > startt) & (df['timestamp'] <= endt)]
mindate = df['timestamp'].dt.date.min()
maxdate = df['timestamp'].dt.date.max()

In [ ]:
# create three nested loops: one for going through the employee id, one for going through each date, and one for going through each clocking session per day
outdf = pd.DataFrame()
ulist = pd.unique(df['Please enter your uNID']).tolist()
ulist.sort()
# loop through the employee ids (unid) and get all the dates associated with each unid
for unid in ulist:
    thisunid = df[df['Please enter your uNID'] == unid]
    thisdlist = pd.unique(thisunid['timestamp'].dt.date)
    # for each date, create a df for keeping all the dates, and a df for dropping dates that have an odd number of punches
    for date in thisdlist:
        thisdate = thisunid[thisunid['timestamp'].dt.date == date].reset_index(drop=True)
        thisdatekeep = thisdate
        daterr = False
        # drop all the dates that only have one punch
        if thisdate['timestamp'].count() == 1:
            thisdate = thisdate.drop(thisdate.tail(1).index)
            daterr = True
        # drop all the dates that have an odd number of punches
        elif thisdate['timestamp'].count()%2 == 1:
            thisdate = thisdate.drop(thisdate.tail(1).index)
            daterr = True
        
        totime = 0
        # loop through each session and evaluate how much time elapsed between punches
        for session in range(int(thisdate['timestamp'].count()/2)):
            sesstime = thisdate['timestamp'][(session*2)+1]-thisdate['timestamp'][(session*2)]
            totime += sesstime.total_seconds()
        totime /= 3600
        # create new df to house the information
        tempdf = pd.DataFrame()
        tempdf['UNID'] = thisdatekeep['Please enter your uNID']
        tempdf['DATE'] = thisdatekeep['timestamp'].dt.date
        tempdf['PUNCHES'] = thisdatekeep['timestamp']
        tempdf['HOURS WORKED'] = totime
        tempdf['PUNCH #'] = thisdatekeep['timestamp'].count()
        if daterr:
            tempdf['PUNCH ERROR'] = thisdatekeep['timestamp'].max()

        outdf = outdf.append(tempdf)

In [ ]:
# we now have our output which lists all the employee ids, the dates they punched in, the times they punched in, and the hours they worked per session.  
# we can see the number of punches they created and a flag to show the dates that had either only one punch or an odd number of punches.
outdf = outdf.reset_index(drop=True)
outdf['PUNCH ERROR'] = outdf['PUNCH ERROR'].replace({pd.NaT: 'NO ERROR'})
outdf.to_csv('Punch_Errors_{}_{}.csv'.format(mindate,maxdate), index=False)
outdf.head(300)
# more code below

,UNID,DATE,PUNCHES,HOURS WORKED,PUNCH #,PUNCH ERROR
0,MISSING,2021-12-16,2021-12-16 11:00:00,0.00,1,2021-12-16 11:00:00
1,u0628352,2021-12-17,2021-12-17 09:15:00,2.25,2,NO ERROR
2,u0628352,2021-12-17,2021-12-17 11:30:00,2.25,2,NO ERROR
3,u0628352,2021-12-22,2021-12-22 08:00:00,8.00,2,NO ERROR
4,u0628352,2021-12-22,2021-12-22 16:00:00,8.00,2,NO ERROR
5,u0657601,2021-12-16,2021-12-16 08:15:00,6.50,4,NO ERROR
6,u0657601,2021-12-16,2021-12-16 13:00:00,6.50,4,NO ERROR
7,u0657601,2021-12-16,2021-12-16 13:30:00,6.50,4,NO ERROR
8,u0657601,2021-12-16,2021-12-16 15:15:00,6.50,4,NO ERROR
9,u0657601,2021-12-17,2021-12-17 09:00:00,5.75,4,NO ERROR


In [ ]:
totaldf = pd.DataFrame()
for unid in ulist:
    thisunid = df[df['Please enter your uNID'] == unid]
    thisdlist = pd.unique(thisunid['timestamp'].dt.date)
    for date in thisdlist:
        thisdate = thisunid[thisunid['timestamp'].dt.date == date].reset_index(drop=True)
        thisdatekeep = thisdate
        daterr = False
        if thisdate['timestamp'].count() == 1:
            thisdate = thisdate.drop(thisdate.tail(1).index)
            daterr = True
        elif thisdate['timestamp'].count()%2 == 1:
            thisdate = thisdate.drop(thisdate.tail(1).index)
            daterr = True
        
        totime = 0
        for session in range(int(thisdate['timestamp'].count()/2)):
            sesstime = thisdate['timestamp'][(session*2)+1]-thisdate['timestamp'][(session*2)]
            totime += sesstime.total_seconds()
        totime /= 3600
        tempdf = pd.DataFrame()
        tempdf['UNID'] = thisunid['Please enter your uNID'].mode()
        tempdf['DATE'] = thisdatekeep['timestamp'].dt.date
        tempdf['HOURS WORKED'] = totime
        tempdf['PUNCH #'] = thisdatekeep['timestamp']
        if daterr:
            tempdf['PUNCH ERROR'] = thisdatekeep['timestamp'].max()

        totaldf = totaldf.append(tempdf)

In [ ]:
totaldf = totaldf.drop(['DATE', 'PUNCH #'], axis=1)

In [ ]:
totaldf = totaldf.groupby('UNID').sum()
totaldf = totaldf.reset_index()
totaldf.to_csv('Total_Summary_{}_{}.csv'.format(mindate,maxdate), index=False)

In [ ]:
# this is a total overview of all the hours worked by each employee
totaldf.head(100)

,UNID,HOURS WORKED
0,MISSING,0.00
1,u0628352,10.25
2,u0657601,40.00
3,u0694788,27.25
4,u0822319,20.00
5,u0879848,14.00
6,u0911145,27.00
7,u0955294,13.25
8,u0960178,24.00
9,u0972427,19.75
